In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

from huggingmolecules import MatModel, MatFeaturizer

sys.path.insert(0, os.path.abspath('..'))

# The following import works only from the source code directory:
from experiments.src import TrainingModule, get_data_loaders

from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam

from pytorch_lightning import Trainer
from pytorch_lightning.metrics import Accuracy
import torch.nn as nn

import numpy as np 
from sklearn.metrics import roc_auc_score, average_precision_score
import wandb
from pytorch_lightning.loggers import WandbLogger  # newline 1
from pytorch_lightning.callbacks import ModelCheckpoint
import torch

In [3]:
import glob
import pandas as pd
datasets = ['bbbp', 'clintox', 'tox21', 'hiv']
data_dir_pattern = "datasets/mymoleculenet/%s/"


In [4]:
for dataset in datasets:
    n_splits = len(glob.glob1(data_dir_pattern%dataset, "[0-9]*"))
    wandb_logger = WandbLogger(name='mat', mode='online', entity='dfstransformer', project='moleculenet10-baselines',
                               config={'dataset':dataset}, settings=wandb.Settings(start_method='fork'))
    rocs = []
    prcs = []
    for rep in range(n_splits):
        # Build and load the pre-trained model and the appropriate featurizer:
        model = MatModel.from_pretrained('mat_masking_20M')
        featurizer = MatFeaturizer.from_pretrained('mat_masking_20M')

        # Build the pytorch lightning training module:
        pl_module = TrainingModule(model,
                                   loss_fn=BCEWithLogitsLoss(),
                                   metric_cls=Accuracy,
                                   optimizer=Adam(model.parameters(), lr=3e-6))
        pl_module.cuda()
        

        trainset = pd.read_csv(data_dir_pattern%dataset+"%d/train.csv"%rep)
        validset = pd.read_csv(data_dir_pattern%dataset+"%d/valid.csv"%rep)
        testset = pd.read_csv(data_dir_pattern%dataset+"%d/test.csv"%rep)
        train_X, train_y = trainset["smiles"].to_numpy(), trainset["target"].to_numpy()
        valid_X, valid_y = validset["smiles"].to_numpy(), validset["target"].to_numpy()
        test_X, test_y = testset["smiles"].to_numpy(), testset["target"].to_numpy()
        
        train_data = featurizer.encode_smiles_list(train_X, train_y)
        valid_data = featurizer.encode_smiles_list(valid_X, valid_y)
        test_data = featurizer.encode_smiles_list(test_X, test_y)
        train_loader = featurizer.get_data_loader(train_data, batch_size=32, shuffle=True, num_workers=12)
        valid_loader = featurizer.get_data_loader(valid_data, batch_size=32, shuffle=False, num_workers=12)
        test_loader = featurizer.get_data_loader(test_data, batch_size=32, shuffle=False, num_workers=12)


        # Build the pytorch lightning trainer and fine-tune the module on the train dataset:
        checkpoint_callback = ModelCheckpoint(monitor="valid_loss", mode="min")
        
        trainer = Trainer(max_epochs=25, gpus=[0], logger=wandb_logger, callbacks=[checkpoint_callback])
        #trainer.tune(pl_module,  train_dataloader=train_loader)
        trainer.fit(pl_module, train_dataloader=train_loader, val_dataloaders=[valid_loader])

        #m = pl_module.eval().cpu()
        m = MatModel.from_pretrained('mat_masking_20M')
        #m.load_state_dict(torch.load(checkpoint_callback.best_model_path)['state_dict'])
        m.load_state_dict({key[6:]:value for key, value in torch.load(checkpoint_callback.best_model_path)['state_dict'].items()}, strict=False)
        m = m.eval().cpu()
        preds = []
        for d in test_loader:
            preds += [m(d).detach().numpy()]
        preds = np.concatenate(preds, axis=0)[:, 0]
        rocs += [roc_auc_score(test_y, preds)]
        prcs += [average_precision_score(test_y, preds)]
        wandb.log({'roc_test':rocs[-1], 'prc_test':prcs[-1]})
    wandb.log({'mean_roc_test':np.mean(rocs), 'std_roc_test':np.std(rocs),
             'mean_prc_test':np.mean(prcs), 'std_prc_test':np.std(rocs)})

RDKit WARNING: [17:27:49] WARNING: not removing hydrogen atom without neighbors
[17:27:49] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:27:49] WARNING: not removing hydrogen atom without neighbors
[17:27:49] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:27:50] WARNING: not removing hydrogen atom without neighbors
[17:27:50] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:27:50] WARNING: not removing hydrogen atom without neighbors
[17:27:50] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:28:12] WARNING: not removing hydrogen atom without neighbors
[17:28:12] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:28:12] WARNING: not removing hydrogen atom without neighbors
[17:28:12] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:28:12] WARNING: not removing hydrogen atom without neighbors
[17:28:12] WARNING: not removing hydrogen atom wit

RDKit WARNING: [17:29:43] WARNING: not removing hydrogen atom without neighbors
[17:29:43] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:29:43] WARNING: not removing hydrogen atom without neighbors
[17:29:43] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:29:58] WARNING: not removing hydrogen atom without neighbors
[17:29:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:29:58] WARNING: not removing hydrogen atom without neighbors
[17:29:58] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:30:00] WARNING: not removing hydrogen atom without neighbors
[17:30:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:30:00] WARNING: not removing hydrogen atom without neighbors
[17:30:00] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:30:02] WARNING: not removing hydrogen atom without neighbors
[17:30:02] WARNING: not removing hydrogen atom wit


  | Name    | Type              | Params
----------------------------------------------
0 | model   | MatModel          | 42 M  
1 | loss_fn | BCEWithLogitsLoss | 0     
INFO:lightning:
  | Name    | Type              | Params
----------------------------------------------
0 | model   | MatModel          | 42 M  
1 | loss_fn | BCEWithLogitsLoss | 0     


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

RDKit ERROR: [02:08:03] UFFTYPER: Unrecognized charge state for atom: 4
[02:08:03] UFFTYPER: Unrecognized charge state for atom: 4
RDKit ERROR: [02:08:03] UFFTYPER: Unrecognized charge state for atom: 4
[02:08:03] UFFTYPER: Unrecognized charge state for atom: 4
RDKit WARNING: [02:08:09] UFFTYPER: Warning: hybridization set to SP3 for atom 6
[02:08:09] UFFTYPER: Warning: hybridization set to SP3 for atom 6
RDKit ERROR: [02:08:09] UFFTYPER: Unrecognized charge state for atom: 6
[02:08:09] UFFTYPER: Unrecognized charge state for atom: 6
RDKit WARNING: [02:08:09] UFFTYPER: Warning: hybridization set to SP3 for atom 6
[02:08:09] UFFTYPER: Warning: hybridization set to SP3 for atom 6
RDKit ERROR: [02:08:09] UFFTYPER: Unrecognized charge state for atom: 6
[02:08:09] UFFTYPER: Unrecognized charge state for atom: 6
RDKit ERROR: [02:08:13] UFFTYPER: Unrecognized charge state for atom: 1
[02:08:13] UFFTYPER: Unrecognized charge state for atom: 1
RDKit ERROR: [02:08:13] UFFTYPER: Unrecognized char

RuntimeError: Pre-condition Violation
	bad params pointer
	Violation occurred on line 75 in file Code/ForceField/UFF/AngleBend.cpp
	Failed Expression: at2Params
	RDKIT: 2020.09.1
	BOOST: 1_74


wandb: Network error (ReadTimeout), entering retry loop.
